In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint

In [9]:
def get_payment_min(s):
    res = ''
    if s.rfind('от')>=0:
        res = ''.join(filter(lambda i: i.isdigit(), s))
        return res
    
    if s.rfind('–')>=0:
        res = s.split('–')[0]
        res = ''.join(filter(lambda i: i.isdigit(), res))
        return res
    
def get_payment_max(s):
    res = ''
    if s.rfind('до')>=0:
        res = ''.join(filter(lambda i: i.isdigit(), s))
        return res
    
    if s.rfind('–')>=0:
        res = s.split('–')[1]
        res = ''.join(filter(lambda i: i.isdigit(), res))
        # print(res)
        return res

In [10]:

# https://rostov.hh.ru/search/vacancy?search_field=name&search_field=company_name&search_field=description&enable_snippets=false&text=Python&from=suggest_post&page=1

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
params = {'enable_snippets': False, 'text': 'Python', 'from': 'suggest_post', 'page': 1}
url = "https://rostov.hh.ru"
session = requests.Session()

articles_list = []


In [11]:
response = session.get(url+'/search/vacancy', headers=headers, params=params)

# print(response.url)
# print(response.status_code)

In [14]:
while response.status_code == 200:
    # if params['page'] == 2:
    #     break

    response = session.get(url+'/search/vacancy', headers=headers, params=params)
    soup = BeautifulSoup(response.text, "html.parser")

    articles = soup.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
    
    for article in articles:
        article_info = {}

        info = article.find('h3')
        name = info.text
        link = info.find('a').get('href')

        b = info.parent
        # payment = b.find_all("span")[1].text.replace('\u202f','')

        payment = b.find_all("span")
        if str(payment).find('<!-- -->')>0:
            payment = payment[1].text.replace('\u202f','')
            payment_min = get_payment_min(payment)
            payment_max = get_payment_max(payment)
            payment_unit = payment[-1]
        else:
            payment = ""
            payment_min = None
            payment_max = None
            payment_unit = None


        
        article_info['name'] = name
        article_info['link'] = link
        article_info['payment_min'] = payment_min
        article_info['payment_max'] = payment_max
        article_info['unit'] = payment_unit
        articles_list.append(article_info)

    print(f"Обработана страница №{params['page']}")
    params['page'] += 1


Обработана страница №1
Обработана страница №2
Обработана страница №3
Обработана страница №4
Обработана страница №5
Обработана страница №6
Обработана страница №7
Обработана страница №8
Обработана страница №9
Обработана страница №10
Обработана страница №11
Обработана страница №12
Обработана страница №13
Обработана страница №14
Обработана страница №15
Обработана страница №16
Обработана страница №17
Обработана страница №18
Обработана страница №19
Обработана страница №20
Обработана страница №21
Обработана страница №22
Обработана страница №23
Обработана страница №24
Обработана страница №25
Обработана страница №26
Обработана страница №27
Обработана страница №28
Обработана страница №29
Обработана страница №30
Обработана страница №31
Обработана страница №32
Обработана страница №33
Обработана страница №34
Обработана страница №35
Обработана страница №36
Обработана страница №37
Обработана страница №38
Обработана страница №39
Обработана страница №40


In [15]:
pprint(articles_list)

[{'link': 'https://rostov.hh.ru/vacancy/83359221?from=vacancy_search_list&query=Python',
  'name': 'Python Developer',
  'payment_max': None,
  'payment_min': '150000',
  'unit': '₽'},
 {'link': 'https://rostov.hh.ru/vacancy/82703691?from=vacancy_search_list&query=Python',
  'name': 'Python developer (FastAPI)',
  'payment_max': '300000',
  'payment_min': None,
  'unit': '₽'},
 {'link': 'https://rostov.hh.ru/vacancy/83080263?from=vacancy_search_list&query=Python',
  'name': 'Разработчик Odoo (Python)',
  'payment_max': None,
  'payment_min': None,
  'unit': None},
 {'link': 'https://rostov.hh.ru/vacancy/82098823?from=vacancy_search_list&query=Python',
  'name': 'Python разработчик',
  'payment_max': None,
  'payment_min': None,
  'unit': None},
 {'link': 'https://rostov.hh.ru/vacancy/83267180?from=vacancy_search_list&query=Python',
  'name': 'Разработчик Python (FastAPI)',
  'payment_max': None,
  'payment_min': '200000',
  'unit': '₽'},
 {'link': 'https://rostov.hh.ru/vacancy/79828667

In [16]:
df = pd.DataFrame({
        "Название вакансии": [],
        "Зарплата от": [],
        "Зарплата до": [],
        "Валюта": [],
        "Ссылка": []
    }) 
n = 0
for i in articles_list:
    n += 1
    df.loc[n] = [i['name'], i['payment_min'], i['payment_max'], i['unit'], i['link']]  # adding a row

df


,Название вакансии,Зарплата от,Зарплата до,Валюта,Ссылка
1,Python Developer,150000,None,₽,https://rostov.hh.ru/vacancy/83359221?from=vac...
2,Python developer (FastAPI),None,300000,₽,https://rostov.hh.ru/vacancy/82703691?from=vac...
3,Разработчик Odoo (Python),None,None,None,https://rostov.hh.ru/vacancy/83080263?from=vac...
4,Python разработчик,None,None,None,https://rostov.hh.ru/vacancy/82098823?from=vac...
5,Разработчик Python (FastAPI),200000,None,₽,https://rostov.hh.ru/vacancy/83267180?from=vac...
...,...,...,...,...,...
776,Аналитик - математик / Data аналитик,None,None,None,https://rostov.hh.ru/vacancy/75823182?from=vac...
777,Team Lead of Analytics,None,None,None,https://rostov.hh.ru/vacancy/82311651?from=vac...
778,DevOps Engineer,None,None,None,https://rostov.hh.ru/vacancy/83319898?from=vac...
779,DevOps инженер,None,None,None,https://rostov.hh.ru/vacancy/83215383?from=vac...


In [17]:
file_name1 = 'vacansions.xlsx'
with pd.ExcelWriter(file_name1) as writer:
    df.to_excel(writer, sheet_name="Список", index=True) 

df.to_csv('vacansions.csv')

